In [ ]:
#Week 3 Assignment
#Part 1

In [24]:
#I'll import everything needed here, even if not of immediate use
import pandas as pd
import requests
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
!pip3 install lxml

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wurl = requests.get(url)


In [4]:
wdata=pd.read_html(wurl.text)
wdata=wdata[0]
mydf=df = wdata[wdata["Borough"] != "Not assigned"]
mydf = mydf.groupby(['Postal Code']).head()
mydf.Neighbourhood.str.count("Not assigned").sum()
mydf = mydf.reset_index()
#and as last the indexes can be dropped
mydf.drop(['index'], axis = 'columns', inplace = True)

In [5]:
mydf.shape

(103, 3)

In [6]:
#Week 3 Assignment
#Part 2

In [7]:
data = pd.read_csv("https://cocl.us/Geospatial_data")

In [8]:
joined = mydf.join(data.set_index('Postal Code'), on='Postal Code', how='inner')
joined

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [9]:
#Week 3 Assignment
#Part 3
#Like in the previous lab, we want to cluster the city of Toronto based on venues categories

In [10]:
!pip install geopy
!pip install folium
from geopy.geocoders import Nominatim
import folium

In [11]:
#some useful data for the creation of the map
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [12]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for latitude, longitude, borough, neighbourhood in zip(joined['Latitude'], joined['Longitude'], joined['Borough'], joined['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_Toronto) 
map_Toronto

In [13]:
CLIENT_ID = '4NHF3YAUIDJJTYH2P2FZXIW0E2FWQTN5HGVCANWVSZ4QRFRI'
CLIENT_SECRET = 'O1F3ZEV3ERFDSHS0DXFBZTUYMHIE0LXI5API112WXKNA5IS1'
VERSION = '20180605' 

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # request to the API
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
t_venues = getNearbyVenues(joined['Neighbourhood'], joined['Latitude'], joined['Longitude'])

In [16]:
t_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,Portuguese Restaurant
...,...,...,...,...,...
1333,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,Tanning Salon
1334,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Once Upon A Child,Kids Store
1335,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Value Village,Thrift / Vintage Store
1336,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Kingsway Boxing Club,Gym


In [17]:
#some df cleaning and organization
toronto_venue = pd.get_dummies(t_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_venue['Neighbourhood'] = t_venues['Neighbourhood'] 
fixed_columns = [toronto_venue.columns[-1]] + list(toronto_venue.columns[:-1])
toronto_venue = toronto_venue[fixed_columns]
toronto_grouped = toronto_venue.groupby('Neighbourhood').mean().reset_index()

In [18]:
#We'll select the top 5 categories of venues to keep everything relatively light (there are many more categories)
def return_most_common_venues(row, top):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:top]

top = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(top):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], top)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Clothing Store,Dessert Shop,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Coffee Shop,Pub,Sandwich Place,Gym,Gay Bar,Donut Shop,Dive Bar,Distribution Center
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Chinese Restaurant,Bridal Shop,Shopping Mall,Sandwich Place,Diner,Deli / Bodega,Restaurant
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Pharmacy,Thai Restaurant,Pizza Place,Pub,Restaurant,Café,Butcher


In [21]:
k_num_clusters = 3
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=k_num_clusters, random_state=0).fit(toronto_grouped_clustering)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [23]:
toronto_merged = joined
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged_nonan = toronto_merged.dropna(subset=['Cluster Labels'])

In [25]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# colors
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_nonan['Latitude'], toronto_merged_nonan['Longitude'], toronto_merged_nonan['Neighbourhood'], toronto_merged_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters)
        
map_clusters

In [26]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 0, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0.0,Park,Food & Drink Shop,Yoga Studio,Department Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Dive Bar
21,York,0.0,Park,Women's Store,Pool,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Dive Bar
35,East York,0.0,Park,Convenience Store,Intersection,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
49,North York,0.0,Park,Construction & Landscaping,Bakery,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
61,Central Toronto,0.0,Park,Swim School,Bus Line,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
64,York,0.0,Park,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Dive Bar
66,North York,0.0,Park,Convenience Store,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
85,Scarborough,0.0,Playground,Park,Bakery,Intersection,Department Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
91,Downtown Toronto,0.0,Park,Tennis Court,Trail,Playground,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


In [27]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 1, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1.0,Intersection,Hockey Arena,Pizza Place,French Restaurant,Coffee Shop,Portuguese Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,Downtown Toronto,1.0,Coffee Shop,Park,Bakery,Breakfast Spot,Theater,French Restaurant,Performing Arts Venue,Chocolate Shop,Pub,Restaurant
3,North York,1.0,Clothing Store,Accessories Store,Boutique,Gift Shop,Furniture / Home Store,Event Space,Coffee Shop,Women's Store,Vietnamese Restaurant,Airport Terminal
4,Downtown Toronto,1.0,Coffee Shop,Yoga Studio,Diner,Sandwich Place,Portuguese Restaurant,Park,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint
6,Scarborough,1.0,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Dive Bar
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,1.0,Café,Coffee Shop,Restaurant,Seafood Restaurant,Gym,Tea Room,Gastropub,Gym / Fitness Center,Hotel,Concert Hall
98,Etobicoke,1.0,Pool,River,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Dive Bar
99,Downtown Toronto,1.0,Coffee Shop,Beer Bar,Breakfast Spot,Bubble Tea Shop,Burger Joint,Salon / Barbershop,Café,Restaurant,Ramen Restaurant,Pub
100,East Toronto,1.0,Recording Studio,Auto Workshop,Skate Park,Park,Light Rail Station,Burrito Place,Farmers Market,Fast Food Restaurant,Butcher,Restaurant


In [28]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 2, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,2.0,Baseball Field,Yoga Studio,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
101,Etobicoke,2.0,Construction & Landscaping,Baseball Field,Yoga Studio,Dim Sum Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop


In [ ]:
#Those where the 3 clusters created